In [1]:
import os
import openai
import pandas as pd
from io import StringIO
from langchain_openai import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
# If you want to chunk PDF text, you can also import TextSplitter utilities:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
import PyPDF2
from dotenv import load_dotenv
import os

## 1. Configure OpenAI Keys

In [2]:
load_dotenv()
OPENAI_API_KEY= os.getenv('OPENAI_API_KEY')
# In code, you might do:
# openai.api_key = os.getenv("OPENAI_API_KEY")

## 2. Helper Functions

In [4]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extract all text from a PDF file using PyPDF2.
    """
    text_content = []
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text_content.append(page.extract_text())
    return "\n".join(text_content)

In [5]:
import pandas as pd
import json

def summarize_excel_to_json(file_path):
    """
    Reads an Excel (.xlsx) file and creates a JSON summary of each sheet.
    For each column it provides:
      1. The variable (column) name.
      2. The inferred variable type (numeric, datetime, categorical).
      3. For numeric columns: standard summary statistics.
      4. For categorical columns: unique values and their counts.
      5. For datetime columns: the time span (min and max dates).

    Parameters:
        file_path (str): The path to the Excel file.

    Returns:
        str: A JSON string summarizing the workbook.
    """
    workbook_summary = {}
    # Load the Excel file
    xls = pd.ExcelFile(file_path)

    # Process each sheet in the workbook
    for sheet in xls.sheet_names:
        df = pd.read_excel(xls, sheet_name=sheet)
        sheet_summary = []
        
        # Process each column in the sheet
        for col in df.columns:
            column_summary = {"variable_name": str(col)}
            series = df[col]

            # Infer the variable type: numeric, datetime, or categorical
            if pd.api.types.is_numeric_dtype(series):
                column_summary["variable_type"] = "numeric"
                # Compute summary statistics
                stats = series.describe()
                column_summary["summary_statistics"] = {
                    "count": stats.get("count"),
                    "mean": stats.get("mean"),
                    "std": stats.get("std"),
                    "min": stats.get("min"),
                    "25%": stats.get("25%"),
                    "50%": stats.get("50%"),
                    "75%": stats.get("75%"),
                    "max": stats.get("max")
                }
            elif pd.api.types.is_datetime64_any_dtype(series):
                column_summary["variable_type"] = "datetime"
                # Compute time span (min and max)
                times = series.dropna()
                if not times.empty:
                    column_summary["time_span"] = {
                        "start": str(times.min()),
                        "end": str(times.max())
                    }
                else:
                    column_summary["time_span"] = {"start": None, "end": None}
            else:
                column_summary["variable_type"] = "categorical"
                # Compute unique values and their counts
                uniques = series.value_counts(dropna=False)
                unique_values = []
                for value, count in uniques.items():
                    # Represent NaN values as the string "NaN"
                    value_str = "NaN" if pd.isna(value) else str(value)
                    unique_values.append({"value": value_str, "count": int(count)})
                column_summary["unique_values"] = unique_values

            sheet_summary.append(column_summary)
        workbook_summary[sheet] = sheet_summary

    # Convert the dictionary to a formatted JSON string
    json_str = json.dumps(workbook_summary, indent=2)
    return json_str

In [7]:
xlsx_path = "../data/poc_example_data/lazaro_et_al_2021.xlsx"
dataset_summary = summarize_excel_to_json(xlsx_path)

## 3. LangChain LLM Setup

In [34]:
summarization_llm = ChatOpenAI(
    temperature=0.0,
    model_name="gpt-4o" 
)

planner_llm = ChatOpenAI(
    model_name="o1-mini"
)

executor_llm = ChatOpenAI(
    model_name="o1-mini"
)

In [32]:
methods_summary_prompt = PromptTemplate(
    input_variables=["full_text"],
    template=(
        "You are a model specialized in ecological data analysis.\n"
        "Read the following text delimited by triple backticks and list all information on the dataset and statistical analyses used.\n"
        "Focus ONLY on:\n"
        "- Extracting all of the tested hypotheses.\n"
        "- Listing all variables that were collected.\n"
        "- Providing example values for categorical variables, and ranges for continuous variables (if available).\n"
        "- Listing all statistical methods with every detail of the performed analysis (specifically which functions, settings, and variables were used).\n"
        "Return only the report.\n"
        "```\n{full_text}\n```"
    )
)


planner_prompt = PromptTemplate(
    input_variables=["methodology_summary", "dataset_summary"],
    template=(
        "You are a planning model. Based on the methodology summary and dataset summary, "
        "plan a step-by-step routine (as programmatic pseudocode or structured steps) "
        "to execute the identified statistical analyses on the full dataset.\n"
        "Think how each analysis helps to answer indyvidual hypotheses.\n\n"
        "Methodology Summary:\n{methodology_summary}\n\n"
        "Dataset Summary:\n{dataset_summary}\n\n"
        "Provide your plan in XML format, with each <Step> containing a structured explanation "
        "of how to implement it programmatically, and which hypothesis it addresses."
    )
)


executor_prompt = PromptTemplate(
    input_variables=["analysis_plan_xml", "dataset_summary"],
    template=(
        "You are an executor model specialized in generating R scripts for each step. "
        "Given the plan in XML, do the following:\n"
        "1. Generate separate R scripts for each analysis step.\n"
        "1.1. Use variable names from the data summary.\n"
        "1.2. Check whether an analysis is suitable for the variable types.\n"
        "1.3. Provide suggestions in comments on alternativees analyses.\n"
        "2. Generate a single master R script that runs them all in a structured manner.\n"
        "Output your results clearly, indicating how the scripts should be saved.\n\n"
        "Plan XML:\n{analysis_plan_xml}.\n"
        "Dataset summary:: \n{dataset_summary}."
    )
)

In [15]:
# --- Step 1: Read PDF and extract methodology ---
pdf_path = "../data/poc_example_data/lazaro_et_al_2021_accessible.pdf"
full_text = extract_text_from_pdf(pdf_path)

In [18]:
chain = methods_summary_prompt | summarization_llm
methods_summary_result = chain.invoke({"full_text": full_text})

In [22]:
# Plan statistical analyses based on the methodology summary and the dataset summary
planner_chain = planner_prompt | planner_llm
plan_analyses_xml = planner_chain.invoke({
    "methodology_summary": methods_summary_result,
    "dataset_summary": dataset_summary
})

In [23]:
plan_analyses_xml

AIMessage(content='```xml\n<AnalysisPlan>\n    <Step number="1">\n        <Description>Load Required Libraries</Description>\n        <Details>\n            <Instruction>Ensure all necessary R packages are installed and loaded. This includes packages for statistical modeling, data manipulation, and network analysis.</Instruction>\n            <Code>\n                <![CDATA[\n                # Install packages if not already installed\n                required_packages <- c("lme4", "MuMIn", "car", "r2glmm", "piecewiseSEM", \n                                       "bipartite", "dplyr", "tidyr", "emmeans", "corrr")\n                installed_packages <- rownames(installed.packages())\n                for(pkg in required_packages){\n                    if(!pkg %in% installed_packages){\n                        install.packages(pkg)\n                    }\n                }\n\n                # Load libraries\n                library(lme4)\n                library(MuMIn)\n                

In [35]:
executor_chain = executor_prompt | executor_llm
executed_plan_steps = executor_chain.invoke({
    "analysis_plan_xml": plan_analyses_xml,
    "dataset_summary": dataset_summary
})

In [40]:
from IPython.display import display, HTML, Markdown
# display(HTML(executed_plan_steps.content))
display(Markdown(executed_plan_steps.content))

### **R Script Generation for Analysis Plan**

Based on the provided **AnalysisPlan** in XML and the **Dataset Summary**, separate R scripts have been generated for each analysis step. Additionally, a master R script has been created to execute all individual scripts in a structured manner.

---

#### **Instructions to Save the Scripts**

1. **Directory Structure:**
   
   - Create a main project directory.
   - Inside the main directory, create a subdirectory named `scripts` to store all individual R scripts.

2. **Saving Individual Scripts:**
   
   - Save each analysis step as a separate `.R` file within the `scripts` folder.
   - Use descriptive filenames corresponding to each analysis step, e.g., `Step1_LoadRequiredLibraries.R`, `Step2_ImportAndInspectDataset.R`, etc.

3. **Master Script:**
   
   - Save the master script as `MasterScript.R` in the main project directory (outside the `scripts` folder).
   - This script will sequentially execute all individual analysis scripts.

---

### **Individual R Scripts**

---

#### **Step 1: Load Required Libraries**

**Filename:** `scripts/Step1_LoadRequiredLibraries.R`

**Description:** Install and load all necessary R packages required for the analysis.

```r
# Step1_LoadRequiredLibraries.R

# Install packages if not already installed
required_packages <- c("lme4", "MuMIn", "car", "r2glmm", "piecewiseSEM", 
                       "bipartite", "dplyr", "tidyr", "emmeans", "corrr", "vegan", "ggplot2", "rmarkdown")
installed_packages <- rownames(installed.packages())
for(pkg in required_packages){
    if(!pkg %in% installed_packages){
        install.packages(pkg, dependencies = TRUE)
    }
}

# Load libraries
library(lme4)
library(MuMIn)
library(car)
library(r2glmm)
library(piecewiseSEM)
library(bipartite)
library(dplyr)
library(tidyr)
library(emmeans)
library(corrr)
library(vegan)
library(ggplot2)
library(rmarkdown)

# Alternative Analyses Suggestions:
# Consider using 'tidyverse' for comprehensive data manipulation.
# For advanced visualization, 'ggplot2' is included, but packages like 'plotly' can offer interactive plots.
```

**Comments:**

- **1.2 Suitability Check:** All listed packages are suitable for statistical modeling, data manipulation, and network analysis as required by the analysis plan.
  
- **1.3 Alternative Suggestions:** Comments have been added to suggest additional packages like `tidyverse` for broader data manipulation capabilities and `plotly` for interactive visualizations.

---

#### **Step 2: Import and Inspect Dataset**

**Filename:** `scripts/Step2_ImportAndInspectDataset.R`

**Description:** Load the dataset into R and perform initial inspections to understand its structure and identify any missing or inconsistent data.

```r
# Step2_ImportAndInspectDataset.R

# Replace 'your_dataset.csv' with the actual file name and path
wild_bee_data <- read.csv("your_dataset.csv", stringsAsFactors = TRUE)

# View the first few rows
head(wild_bee_data)

# Summary statistics
summary(wild_bee_data)

# Check for missing values
sapply(wild_bee_data, function(x) sum(is.na(x)))

# Alternative Analyses Suggestions:
# Use 'glimpse(wild_bee_data)' from dplyr for a transposed view of the data structure.
# Utilize 'skimr::skim(wild_bee_data)' for a more detailed summary (requires installing 'skimr').
```

**Comments:**

- **1.2 Suitability Check:** The dataset is being read with `stringsAsFactors = TRUE`, which is appropriate for categorical variables as per the data summary. Ensure that all categorical variables are correctly identified.
  
- **1.3 Alternative Suggestions:** Comments suggest using `glimpse` for an alternative data overview and `skimr::skim` for a more comprehensive summary, enhancing data inspection.

---

#### **Step 3: Data Preprocessing**

**Filename:** `scripts/Step3_DataPreprocessing.R`

**Description:** Prepare the data for analysis by handling missing values, encoding categorical variables, and scaling numerical variables as required.

```r
# Step3_DataPreprocessing.R

# Load necessary libraries
library(dplyr)

# 3.1 Handle Missing Values
# Strategy: Remove rows with missing values
wild_bee_data <- na.omit(wild_bee_data)

# Alternative Suggestions:
# Consider multiple imputation using the 'mice' package if data loss from omission is a concern.

# 3.2 Encode Categorical Variables
# Convert categorical variables to factors using variable names from the data summary
categorical_vars <- c("Island", "Island_Site", "family", "InsectSpecies", "Apis")
wild_bee_data[categorical_vars] <- lapply(wild_bee_data[categorical_vars], as.factor)

# Alternative Suggestions:
# Use 'forcats' package for more nuanced factor manipulation, such as releveling or collapsing categories.

# 3.3 Scale Numerical Variables
# Define numerical variables based on the data summary
numeric_vars <- c("NumberHivesMinistry2005_2015", 
                  "HiveDensityMinistry2005-2015",
                  "HiveDensityMinistry_STUDYYEAR", 
                  "NumberApis", 
                  "FlowerAbundance",
                  "FlowerRichness", 
                  "LandscapeHeterogeneity", 
                  "Areakm2",
                  "%NaturalHabitats")

# Scale numerical predictors
wild_bee_data[numeric_vars] <- scale(wild_bee_data[numeric_vars])

# Alternative Suggestions:
# Explore robust scaling methods using the 'scale' function with parameters or using the 'scale' function from 'caret'.

# Additional Check:
# Verify the scaling
summary(wild_bee_data[numeric_vars])
```

**Comments:**

- **1.2 Suitability Check:** 
  - **Handling Missing Values:** The chosen strategy (`na.omit`) is suitable given the dataset size. However, alternative strategies like imputation might preserve more data.
  - **Encoding Categorical Variables:** Converting specified variables to factors aligns with their types in the data summary.
  - **Scaling Numerical Variables:** Scaling is appropriate for models sensitive to variable scaling (e.g., GLMMs).

- **1.3 Alternative Suggestions:** Comments suggest alternative packages and methods for imputation, factor manipulation, and scaling to enhance preprocessing flexibility.

---

#### **Step 4: Generalized Linear Mixed Models (GLMMs)**

**Filename:** `scripts/Step4_GLMMS.R`

**Description:** Analyze the relationship between honey bee visitation rate and wild bee abundance and richness using GLMMs.

```r
# Step4_GLMMS.R

# Load necessary libraries
library(lme4)
library(MuMIn)
library(car)
library(r2glmm)

# 4.1 Model Wild Bee Abundance
# Fit a GLMM for wild bee abundance with honey bee visitation rate and other predictors.
# Include 'Island' as a random effect and use a Gamma distribution with a log link function.

abundance_model <- glmer(WildBeeAbundanceSt ~ NumberApis + FlowerAbundance + FlowerRichness + 
                             LandscapeHeterogeneity + log(Areakm2) + 
                             (1 | Island), 
                         data = wild_bee_data, 
                         family = Gamma(link = "log"))

summary(abundance_model)

# Alternative Analyses Suggestions:
# If the Gamma distribution is not appropriate, consider a Poisson or Negative Binomial distribution for count data.

# 4.2 Model Wild Bee Richness
# Fit a GLMM for wild bee richness using the same predictors and random effect as above.

richness_model <- glmer(WildBeeRichnessSt ~ NumberApis + FlowerAbundance + FlowerRichness + 
                            LandscapeHeterogeneity + log(Areakm2) + 
                            (1 | Island), 
                        data = wild_bee_data, 
                        family = Gamma(link = "log"))

summary(richness_model)

# Alternative Analyses Suggestions:
# For richness data, a Poisson or Zero-Inflated model might be more suitable if overdispersion is an issue.

# 4.3 Model Selection with Dredge
# Perform model selection using the dredge function to identify the best models with up to 4 predictors.

# Set global options for MuMIn
options(na.action = "na.fail")

# Model selection for abundance
abundance_dredge <- dredge(abundance_model, m.lim = c(0,4))
print(abundance_dredge)

# Select the best model
best_abundance_model <- get.models(abundance_dredge, subset = 1)[[1]]
summary(best_abundance_model)

# Model selection for richness
richness_dredge <- dredge(richness_model, m.lim = c(0,4))
print(richness_dredge)

# Select the best model
best_richness_model <- get.models(richness_dredge, subset = 1)[[1]]
summary(best_richness_model)

# Alternative Analyses Suggestions:
# Consider using AICc for model selection or other criteria like BIC based on research priorities.
```

**Comments:**

- **1.2 Suitability Check:** 
  - **GLMMs for Abundance and Richness:** Both models are appropriately specified for continuous dependent variables using the Gamma family. Ensure that the Gamma distribution fits the data well; check residuals and dispersion.
  
- **1.3 Alternative Suggestions:** 
  - For both models, alternative distributions like Poisson, Negative Binomial, or Zero-Inflated models are suggested depending on the data's characteristics (e.g., count data, overdispersion).

---

#### **Step 5: Variation Inflation Factor (VIF) Analysis**

**Filename:** `scripts/Step5_VIFAnalysis.R`

**Description:** Identify and address multicollinearity among landscape variables using VIF.

```r
# Step5_VIFAnalysis.R

# Load necessary libraries
library(car)

# 5.1 Calculate VIF for the best abundance model
vif_abundance <- vif(best_abundance_model)
print(vif_abundance)

# 5.2 Calculate VIF for the best richness model
vif_richness <- vif(best_richness_model)
print(vif_richness)

# 5.3 Interpret VIF results
# Typically, VIF > 5 or 10 indicates multicollinearity issues.

if(any(vif_abundance > 5)){
    warning("High VIF detected in abundance model. Consider removing or combining variables.")
}

if(any(vif_richness > 5)){
    warning("High VIF detected in richness model. Consider removing or combining variables.")
}

# Alternative Analyses Suggestions:
# Consider using Principal Component Analysis (PCA) to reduce dimensionality if multicollinearity is present.
```

**Comments:**

- **1.2 Suitability Check:** VIF is an appropriate method to detect multicollinearity among predictor variables in the models.
  
- **1.3 Alternative Suggestions:** Suggests PCA as an alternative method to handle multicollinearity by reducing the number of predictors.

---

#### **Step 6: Piecewise Structural Equation Modeling (SEM)**

**Filename:** `scripts/Step6_PiecewiseSEM.R`

**Description:** Summarize direct and indirect effects of honey bee visitation rate on wild bee network metrics using Piecewise SEM.

```r
# Step6_PiecewiseSEM.R

# Load necessary libraries
library(piecewiseSEM)
library(lme4)

# 6.1 Scale Variables
wild_bee_data_scaled <- wild_bee_data %>%
    mutate_at(vars(NumberApis, FlowerAbundance, FlowerRichness, 
                   LandscapeHeterogeneity, Areakm2, 
                   WildBeeAbundanceSt, WildBeeRichnessSt), scale)

# 6.2 Define SEM Model
sem_model <- psem(
    glmer(WildBeeAbundanceSt ~ NumberApis + FlowerAbundance + FlowerRichness + 
              LandscapeHeterogeneity + log(Areakm2) + 
              (1 | Island), 
          data = wild_bee_data_scaled, 
          family = Gamma(link = "log")),
    
    glmer(WildBeeRichnessSt ~ NumberApis + FlowerAbundance + FlowerRichness + 
              LandscapeHeterogeneity + log(Areakm2) + 
              (1 | Island), 
          data = wild_bee_data_scaled, 
          family = Gamma(link = "log")),
    
    lm(log(Areakm2) ~ NumberApis + FlowerAbundance + 
              FlowerRichness + LandscapeHeterogeneity, 
       data = wild_bee_data_scaled),
    
    lm(NumberApis ~ FlowerAbundance + FlowerRichness + LandscapeHeterogeneity, 
       data = wild_bee_data_scaled)
)

# 6.3 Fit SEM Model and Assess Fit
sem_fit <- sem.fit(sem_model, data = wild_bee_data_scaled)
summary(sem_fit, standardize = TRUE, fit.measures = TRUE)

# Goodness-of-fit tests
sem_fit_standard <- sem.fit(sem_model, data = wild_bee_data_scaled, test = "standard")
summary(sem_fit_standard)

# Alternative Analyses Suggestions:
# Explore alternative model specifications or include additional latent variables if theoretical justification exists.
```

**Comments:**

- **1.2 Suitability Check:** 
  - **Scaling Variables:** Scaling is appropriately performed for variables to ensure model convergence and interpretability.
  - **SEM Specification:** The model correctly specifies relationships among variables, aligning with the dataset's structure.

- **1.3 Alternative Suggestions:** 
  - Encourages exploration of alternative SEM specifications and the inclusion of latent variables where appropriate.

---

#### **Step 7: Pollination Network Analysis**

**Filename:** `scripts/Step7_PollinationNetworkAnalysis.R`

**Description:** Construct and analyze pollination networks using the `bipartite` package.

```r
# Step7_PollinationNetworkAnalysis.R

# Load necessary libraries
library(bipartite)

# 7.1 Construct Bipartite Networks
# Replace 'pollination_network.csv' with the actual file name and path
pollination_data <- read.csv("pollination_network.csv", stringsAsFactors = TRUE)

# Create a bipartite matrix
bipartite_matrix <- with(pollination_data, table(PlantSpecies, PollinatorSpecies))

# Convert to matrix
bipartite_matrix <- as.matrix(bipartite_matrix)

# Alternative Analyses Suggestions:
# For more detailed network structures, consider using graph-based packages like 'igraph'.

# 7.2 Calculate Network Indices

# Number of links
num_links <- networklevel(bipartite_matrix, index = "links")

# Number of plant species
num_plants <- networklevel(bipartite_matrix, index = "species.level")

# Linkage density
linkage_density <- linkage(bipartite_matrix)

# Nestedness
nestedness <- networklevel(bipartite_matrix, index = "weighted NODF")

# Modularity
modularity <- networklevel(bipartite_matrix, index = "module")

# Combine indices into a dataframe
network_indices <- data.frame(
    NumberOfLinks = num_links,
    NumberOfPlantSpecies = num_plants,
    LinkageDensity = linkage_density,
    Nestedness = nestedness,
    Modularity = modularity
)

# 7.3 Assess Against Null Models
# Calculate z-scores using the 'vaznull' null model
null_model <- NULLMODEL(bipartite_matrix, method = "vaznull")

z_linkage_density <- null_model$zLinkage
z_nestedness <- null_model$zNODF
z_modularity <- null_model$zModularity

# Add z-scores to network_indices
network_indices$Z_LinkageDensity <- z_linkage_density
network_indices$Z_Nestedness <- z_nestedness
network_indices$Z_Modularity <- z_modularity

# Alternative Analyses Suggestions:
# Explore other null model methods provided by the 'bipartite' package to validate robustness.
```

**Comments:**

- **1.2 Suitability Check:** 
  - **Bipartite Networks:** Suitable for analyzing plant-pollinator interactions as per the dataset.
  
- **1.3 Alternative Suggestions:** 
  - Suggests using `igraph` for more complex network analyses, providing flexibility beyond bipartite matrices.

---

#### **Step 8: Potential Competition Analysis**

**Filename:** `scripts/Step8_PotentialCompetitionAnalysis.R`

**Description:** Measure species niche overlap using the Apparent Competition Index as per Müller et al. (1999).

```r
# Step8_PotentialCompetitionAnalysis.R

# Load necessary libraries
library(vegan)

# 8.1 Prepare Species Interaction Matrix
# Transpose the bipartite matrix for alignment
transposed_matrix <- t(bipartite_matrix)

# Ensure the matrix is binary (presence/absence)
transposed_matrix[transposed_matrix > 0] <- 1

# Alternative Analyses Suggestions:
# Use other niche overlap indices like Pianka’s or Schoener’s for comparative analysis.

# 8.2 Calculate Apparent Competition Index (PAC)
# Note: Specific PAC calculation requires defined formula. Using nicheOverlap as a proxy.

# Calculate niche overlap using Pianka's index
niche_overlap <- nicheOverlap(transposed_matrix, method = "Pianka")

# Placeholder for Apparent Competition Index if specific formula is needed
# Example PAC calculation (modify as per Müller et al. (1999) formula):
# PAC <- (sum of shared resources) / (sum of total resources per species)

# Since PAC is specific, ensure the correct formula is implemented here based on the study.
# Alternatively, consult relevant ecological packages or literature for precise implementation.

# Alternative Analyses Suggestions:
# If PAC calculation is complex, consider consulting specialized ecological analysis packages or scripts.
```

**Comments:**

- **1.2 Suitability Check:** 
  - **Niche Overlap Calculation:** Using `nicheOverlap` from the `vegan` package is appropriate for assessing species niche overlap, aligning with PAC objectives.
  
- **1.3 Alternative Suggestions:** 
  - Suggests exploring other indices and ensuring the specific PAC formula is correctly implemented based on the referenced study.

---

#### **Step 9: A Posteriori Tests for Differences Between Slopes**

**Filename:** `scripts/Step9_APosterioriTests.R`

**Description:** Perform post-hoc tests to examine differences between slopes for each wild bee family using the `emtrends` function from the `emmeans` package.

```r
# Step9_APosterioriTests.R

# Load necessary libraries
library(lme4)
library(emmeans)

# 9.1 Fit Interaction Models

# Interaction model for abundance
interaction_model_abundance <- glmer(WildBeeAbundanceSt ~ NumberApis * family + FlowerAbundance + 
                                         FlowerRichness + LandscapeHeterogeneity + log(Areakm2) + 
                                         (1 | Island), 
                                     data = wild_bee_data, 
                                     family = Gamma(link = "log"))

# Interaction model for richness
interaction_model_richness <- glmer(WildBeeRichnessSt ~ NumberApis * family + FlowerAbundance + 
                                        FlowerRichness + LandscapeHeterogeneity + log(Areakm2) + 
                                        (1 | Island), 
                                    data = wild_bee_data, 
                                    family = Gamma(link = "log"))

# 9.2 Perform Emtrends Analysis

# Emtrends for abundance
emtrends_abundance <- emtrends(interaction_model_abundance, 
                              pairwise ~ family, 
                              var = "NumberApis")
summary(emtrends_abundance)

# Emtrends for richness
emtrends_richness <- emtrends(interaction_model_richness, 
                             pairwise ~ family, 
                             var = "NumberApis")
summary(emtrends_richness)

# Alternative Analyses Suggestions:
# Consider using `multcomp` package for multiple comparisons if alternative methods are preferred.
```

**Comments:**

- **1.2 Suitability Check:** 
  - **Interaction Models:** Appropriately include interaction terms between `NumberApis` and `family` to assess slope differences.
  
- **1.3 Alternative Suggestions:** 
  - Suggests using the `multcomp` package as an alternative for performing multiple comparisons.

---

#### **Step 10: Export and Document Results**

**Filename:** `scripts/Step10_ExportAndDocumentResults.R`

**Description:** Save all model outputs, summaries, and important statistics to files for reporting and further analysis.

```r
# Step10_ExportAndDocumentResults.R

# 10.1 Save Model Summaries

# Save GLMM summaries
sink("GLMM_Abundance_Summary.txt")
summary(best_abundance_model)
sink()

sink("GLMM_Richness_Summary.txt")
summary(best_richness_model)
sink()

# Save SEM summary
sink("SEM_Summary.txt")
summary(sem_fit, standardize = TRUE, fit.measures = TRUE)
sink()

# Save Emtrends summaries
sink("Emtrends_Abundance_Summary.txt")
summary(emtrends_abundance)
sink()

sink("Emtrends_Richness_Summary.txt")
summary(emtrends_richness)
sink()

# Alternative Analyses Suggestions:
# Consider exporting summaries in HTML or PDF formats for better readability using R Markdown.

# 10.2 Save Network Indices
write.csv(network_indices, "Pollination_Network_Indices.csv", row.names = FALSE)

# 10.3 Save PAC Results
write.csv(niche_overlap, "Apparent_Competition_Index.csv", row.names = TRUE)

# Alternative Analyses Suggestions:
# Use `saveRDS` for saving R objects if further R-based analysis is required.
```

**Comments:**

- **1.2 Suitability Check:** 
  - **Exporting Results:** The chosen methods (`sink`, `write.csv`) are appropriate for exporting textual summaries and dataframes.

- **1.3 Alternative Suggestions:** 
  - Suggests using R Markdown for more formatted and readable report exports and `saveRDS` for preserving R objects.

---

#### **Step 11: Visualization of Results**

**Filename:** `scripts/Step11_VisualizationOfResults.R`

**Description:** Create visualizations to illustrate the findings from the statistical analyses, such as effect plots, network diagrams, and VIF plots.

```r
# Step11_VisualizationOfResults.R

# Load necessary libraries
library(emmeans)
library(ggplot2)
library(bipartite)

# 11.1 Effect Plots for GLMMs

# Plot for abundance
plot(emtrends_abundance, comparisons = TRUE) +
    ggtitle("Effect of NumberApis on Wild Bee Abundance by Family") +
    theme_minimal()

# Save the plot
ggsave("EffectPlot_Abundance.png")

# Plot for richness
plot(emtrends_richness, comparisons = TRUE) +
    ggtitle("Effect of NumberApis on Wild Bee Richness by Family") +
    theme_minimal()

# Save the plot
ggsave("EffectPlot_Richness.png")

# Alternative Analyses Suggestions:
# Enhance plots with custom themes or interactivity using packages like `plotly`.

# 11.2 Network Diagrams

# Basic network plot
plotweb(bipartite_matrix, method = "normal", 
        title = "Pollination Network", 
        text.rot = 90, text.spacing = 0.5)

# Save the plot
ggsave("Pollination_Network.png")

# Alternative Analyses Suggestions:
# Use `igraph` for more customizable and interactive network visualizations.

# 11.3 VIF Plot

# Calculate VIF
vif_values <- vif(best_abundance_model)
vif_df <- data.frame(Variable = names(vif_values), VIF = vif_values)

# Create VIF plot
vif_plot <- ggplot(vif_df, aes(x = reorder(Variable, VIF), y = VIF)) +
    geom_bar(stat = "identity", fill = "steelblue") +
    coord_flip() +
    geom_hline(yintercept = 5, linetype = "dashed", color = "red") +
    labs(title = "VIF Values for Predictors",
         x = "Variables",
         y = "VIF") +
    theme_minimal()

# Display the plot
print(vif_plot)

# Save the plot
ggsave("VIF_Plot.png", plot = vif_plot)

# Alternative Analyses Suggestions:
# Annotate the plot with labels for VIF values or use color gradients to indicate severity.
```

**Comments:**

- **1.2 Suitability Check:** 
  - **Visualization Techniques:** The selected plotting methods (`ggplot2`, `plotweb`) are suitable for visualizing model effects, network structures, and VIF values.

- **1.3 Alternative Suggestions:** 
  - Suggests enhancing plots with additional features or interactivity for improved interpretability and presentation.

---

#### **Step 12: Report Findings**

**Filename:** `scripts/Step12_ReportFindings.R`

**Description:** Compile all results, figures, and interpretations into a comprehensive report detailing the impacts of beekeeping on wild bee diversity and pollination networks.

```r
# Step12_ReportFindings.R

# Load necessary library
library(rmarkdown)

# 12.1 Create Report Structure
# Create a new R Markdown file with necessary sections
rmarkdown::draft("Bee_Analysis_Report.Rmd", template = "html_document", package = "rmarkdown")

# 12.2 Incorporate Results and Figures
# Instructions (to be performed manually in the R Markdown file):
# - Insert code chunks to read and display saved summaries.
# - Example code chunks to include in 'Bee_Analysis_Report.Rmd':
#
# ```{r GLMM_Summary}
# cat(readLines("GLMM_Abundance_Summary.txt"), sep = "\n")
# ```
#
# ```{r Network_Indices}
# network_indices <- read.csv("Pollination_Network_Indices.csv")
# knitr::kable(network_indices)
# ```
#
# ```{r VIF_Plot, fig.height=6, fig.width=8}
# vif_plot <- ggplot(vif_df, aes(x = reorder(Variable, VIF), y = VIF)) +
#     geom_bar(stat = "identity", fill = "steelblue") +
#     coord_flip() +
#     geom_hline(yintercept = 5, linetype = "dashed", color = "red") +
#     labs(title = "VIF Values for Predictors",
#          x = "Variables",
#          y = "VIF") +
#     theme_minimal()
# print(vif_plot)
# ```
#
# - Similarly, include other summaries and plots as needed.

# 12.3 Finalize and Knit Report
# After manually incorporating all necessary content, render the R Markdown report.

# Knit the report to HTML format
rmarkdown::render("Bee_Analysis_Report.Rmd")

# Alternative Analyses Suggestions:
# Customize the R Markdown template to match the desired report aesthetics and structure.
```

**Comments:**

- **1.2 Suitability Check:** 
  - **Report Generation:** Using R Markdown is suitable for compiling a comprehensive report that integrates text, code, and visualizations.
  
- **1.3 Alternative Suggestions:** 
  - Suggests customizing the R Markdown template to better fit specific reporting needs and aesthetics.

---

### **Master R Script**

**Filename:** `MasterScript.R`

**Description:** Master script to execute all individual analysis scripts in a structured manner.

```r
# MasterScript.R

# Set working directory to the project folder
# setwd("path_to_your_project_folder")  # Uncomment and set your project directory path

# Source individual scripts in order
source("scripts/Step1_LoadRequiredLibraries.R")
source("scripts/Step2_ImportAndInspectDataset.R")
source("scripts/Step3_DataPreprocessing.R")
source("scripts/Step4_GLMMS.R")
source("scripts/Step5_VIFAnalysis.R")
source("scripts/Step6_PiecewiseSEM.R")
source("scripts/Step7_PollinationNetworkAnalysis.R")
source("scripts/Step8_PotentialCompetitionAnalysis.R")
source("scripts/Step9_APosterioriTests.R")
source("scripts/Step10_ExportAndDocumentResults.R")
source("scripts/Step11_VisualizationOfResults.R")
source("scripts/Step12_ReportFindings.R")

# Inform the user that all scripts have been run
cat("All analysis scripts have been successfully executed.\n")
```

**Comments:**

- **Execution Flow:** The master script sequentially sources each individual analysis script, ensuring that all steps are performed in the correct order.
  
- **Customization:** Users should set their working directory appropriately by uncommenting and modifying the `setwd()` function as needed.

---

### **Summary of Script Saving**

1. **Create Project Directory:**
   
   - Choose a suitable location on your computer and create a new folder for the project, e.g., `Bee_Analysis_Project`.

2. **Create Scripts Subdirectory:**
   
   - Inside `Bee_Analysis_Project`, create a subfolder named `scripts`.

3. **Save Individual Scripts:**
   
   - Save each of the individual `.R` scripts (Steps 1 to 12) into the `scripts` folder with the filenames as specified above.

4. **Save Master Script:**
   
   - Save the `MasterScript.R` file directly inside `Bee_Analysis_Project`.

5. **Run the Master Script:**
   
   - Open R or RStudio, set the working directory to `Bee_Analysis_Project`, and run `MasterScript.R` to execute the entire analysis pipeline.

---

### **Final Notes**

- **Dependencies:** Ensure that all necessary packages are installed and up-to-date before running the scripts.

- **Data Files:** Replace placeholder filenames like `your_dataset.csv` and `pollination_network.csv` with the actual paths to your data files.

- **Customization:** Review and modify the scripts as needed to better fit the specific nuances of your data and research questions.

- **Error Handling:** Monitor the console for any warnings or errors during script execution and address them accordingly.

By following this structured approach, you can efficiently perform comprehensive analyses on the impacts of beekeeping on wild bee diversity and pollination networks, culminating in a well-documented report of your findings.

## Refine each step and run it